In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset,DataLoader
import numpy as np
from scipy import interpolate
import random
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    #device = torch.device("cpu")
else:
    device = torch.device("cpu")
m=900
Nnn=800+800#total number of samples
heat_up_number=10#number of warmback stages
os.chdir('/kaggle/input/info-combine-0225')
y_mm_2=np.array(pd.read_csv('position_heatup_train_2frac_iso_0225.csv'))#fracture labels
T_inj_2=np.array(pd.read_csv('T_injection_train_2frac_iso_0225.csv')).reshape(800,m+1,22)[:,:,0]#wellbore temperature at the end of injection
T_static_2=np.array(pd.read_csv('2fracture_T_heatup_multistage_New_type_geo_initial_for_0225.csv')).reshape(800,m+1)#static stage temperature
XX_2= np.array(pd.read_csv('2fracture_T_heatup_multistage_0225.csv'))#wellbore temperature during warmback
real_position_2=np.array(pd.read_csv('2fracture_multistage_realposition_0225.csv'))
XX_2=XX_2.reshape(800,m+1,heat_up_number)
XX_2=XX_2.transpose(0,2,1)
real_position_2=real_position_2.reshape(800,m+1,heat_up_number)
real_position_2=real_position_2.transpose(0,2,1)
os.chdir('/kaggle/input/inj-warmback-static-0216')
y_mm_3=np.array(pd.read_csv('position_heatup_train_2frac_iso_0219.csv'))#(number_samples=100,m=901)
T_inj_3=np.array(pd.read_csv('T_injection_train_2frac_iso_0219.csv')).reshape(800,m+1,22)[:,:,0]
T_static_3=np.array(pd.read_csv('2fracture_T_heatup_multistage_New_type_geo_initial_for_0219.csv')).reshape(800,m+1)
XX_3= np.array(pd.read_csv('2fracture_T_heatup_multistage_0219.csv'))
real_position_3=np.array(pd.read_csv('2fracture_multistage_realposition_0219.csv'))
Nnn2=800
XX_3=XX_3.reshape(Nnn2,m+1,heat_up_number)
XX_3=XX_3.transpose(0,2,1)
real_position_3=real_position_3.reshape(Nnn2,m+1,heat_up_number)
real_position_3=real_position_3.transpose(0,2,1)

XX=np.zeros((Nnn-2,heat_up_number,m+1))
TT_inj=np.zeros((Nnn-2,m+1))
TT_static=np.zeros((Nnn-2,m+1))
XX=torch.cat((torch.Tensor(XX_3),torch.Tensor(XX_2[0:435,:,:]),torch.Tensor(XX_2[436:777,:,:]),torch.Tensor(XX_2[778:,:,:])),dim=0)
XX=XX.numpy()
XX=XX.reshape((Nnn-2)*heat_up_number,m+1)

TT_inj=torch.cat((torch.Tensor(T_inj_3),torch.Tensor(T_inj_2[:435,:]),torch.Tensor(T_inj_2[436:777,:]),torch.Tensor(T_inj_2[778:,:])),dim=0)
TT_inj=TT_inj.numpy()
TT_static=torch.cat((torch.Tensor(T_static_3),torch.Tensor(T_static_2[:435,:]),torch.Tensor(T_static_2[436:777,:]),torch.Tensor(T_static_2[778:,:])),dim=0)
TT_static=TT_static.numpy()

real_position=np.zeros((Nnn-2,heat_up_number,m+1))
real_position=torch.cat((torch.Tensor(real_position_3),torch.Tensor(real_position_2[:435,:,:]),torch.Tensor(real_position_2[436:777,:,:]),torch.Tensor(real_position_2[778:,:,:])),dim=0)
real_position=real_position.numpy()
real_position=real_position.reshape((Nnn-2)*heat_up_number,m+1)
y_mm=np.zeros((Nnn-2,m+1))
y_mm=torch.cat((torch.Tensor(y_mm_3),torch.Tensor(y_mm_2[:435,:]),torch.Tensor(y_mm_2[436:777,:]),torch.Tensor(y_mm_2[778:,:])),dim=0)                         
y_mm=y_mm.numpy()
#interpolate
XX_new=np.zeros(((Nnn-2)*heat_up_number,m+1))
real_position_new=np.zeros(((Nnn-2)*heat_up_number,m+1))
TT_inj_new=np.zeros(((Nnn-2),m+1))
TT_static_new=np.zeros(((Nnn-2),m+1))
for i in range(0,(Nnn-2)*heat_up_number):
    real_position_new[i,0:895]=np.arange(0,895)
    real_position_new[i,895]=894.5
    real_position_new[i,896]=895
    real_position_new[i,897]=895.5
    real_position_new[i,898]=896
    real_position_new[i,899]=896.5
    real_position_new[i,900]=897
    real_position_new[i,0]=1.05
for i in range(0,(Nnn-2)*heat_up_number):
    f = interpolate.interp1d(real_position[i,:], XX[i,:])
    XX_new[i,:]=f(real_position_new[i,:]) 
XX_new=XX_new.reshape((Nnn-2),heat_up_number,m+1)
#normalize temp, every sample for all heatup stages uses the same max and min
for i in range(Nnn-2):
    XX_new[i,:,:]=(XX_new[i,:,:]-np.min(XX_new[i,:,:]))/(np.max(XX_new[i,:,:])-np.min(XX_new[i,:,:]))
real_position_new=real_position_new.reshape((Nnn-2),heat_up_number,m+1)
#for injection data interpolation
real_position=real_position.reshape((Nnn-2),heat_up_number,m+1)
for j in range(0,Nnn-2):
    ff = interpolate.interp1d(real_position[j,0,:], TT_inj[j,:])
    fff=interpolate.interp1d(real_position[j,0,:], TT_static[j,:])
    TT_inj_new[j,:]=ff(real_position_new[j,0,:]) 
    TT_static_new[j,:]=fff(real_position_new[j,0,:]) 
for i in range(Nnn-2):
    TT_inj_new[i,:]=(TT_inj_new[i,:]-np.min(TT_inj_new[i,:]))/(np.max(TT_inj_new[i,:])-np.min(TT_inj_new[i,:])) 
    TT_static_new[i,:]=(TT_static_new[i,:]-np.min(TT_static_new[i,:]))/(np.max(TT_static_new[i,:])-np.min(TT_static_new[i,:]))        
XXXX=np.zeros((Nnn-2,heat_up_number+2,len(XX_new[0,0,:])))#add inj replace xx_new
XXXX[:,0,:]=TT_static_new#first input feature: single static stage 
XXXX[:,1,:]=TT_inj_new#second input feature: single injection stage
XXXX[:,2:,:]=XX_new# rest input features: warmback stages
Index=random.sample(range(0,Nnn-2),Nnn-2-200)
X_train=np.zeros((Nnn-2-200,heat_up_number+2,len(XX_new[0,0,:])))
y_train=np.zeros((Nnn-2-200,len(XX_new[0,0,:])))
X_test=[]
y_test=[]
for j in range(0,Nnn-2-200):
    X_train[j]=XXXX[Index[j],:,:]
    y_train[j]=y_mm[Index[j],:]
aa=list(np.arange(0,Nnn-2))
cc=[x for x in aa if x in Index]
dd=[y for y in (aa+list(Index))if y not in cc]
for h in range(len(dd)):
    XXXX[dd[h],:,:]=XXXX[dd[h],:,:]
    X_test.append(XXXX[dd[h],:,:])
    y_test.append(y_mm[dd[h],:])
y_train_tensors = torch.LongTensor(y_train)
y_test_tensors = torch.LongTensor(np.array(y_test))
#add noise to input
noise_std_input = 0.01
X_train_noisy = X_train + noise_std_input * np.random.randn(*np.array(X_train).shape)
X_test_noisy=X_test + noise_std_input * np.random.randn(*np.array(X_test).shape)
X_train_tensors=torch.Tensor(X_train_noisy[:,2:8,:])
X_test_tensors=torch.Tensor(X_test_noisy[:,2:8,:])
class TemDataset(Dataset):
    def __init__(self,X_train_tensors,y_train_tensors):
        self.x=X_train_tensors
        self.y=y_train_tensors
        self.n_samples=self.x.shape[0]
    
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.n_samples
dataset=TemDataset(X_train_tensors,y_train_tensors)
dataloader=DataLoader(dataset=dataset,batch_size=256,shuffle=True)#444
print("Training Shape", X_train_tensors.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors.shape, y_test_tensors.shape)

In [ ]:
#generate figure 9 in the paper(may be different with the figure in paper because of the randomly chosen training set and the random generated noise)
plt.plot(X_train_noisy[399,1,:],-real_position_new[5,1,:],"r-",label="Injection",linewidth=0.7)
plt.plot(X_train_noisy[399,4,10:],-real_position_new[5,1,10:],"g-.",label="Warmback",linewidth=0.7)
plt.plot(X_train_noisy[399,0,:],-real_position_new[5,1,:],"b--",label="Static",linewidth=0.7)
plt.xlabel("Dimensionless wellbore temperature/[-]")
plt.ylabel("Depth/m")
plt.legend()

In [ ]:
Num_feature=6#need to change, depends on how many input features used
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length,embed_size=40*2,heads=1,drop_prob=0):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        self.embed_size=embed_size
        self.heads=heads
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,bidirectional=True,dropout=drop_prob) #lstm
        self.gru=nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,dropout=drop_prob)
        self.multiattention=nn.MultiheadAttention(embed_size,heads,dropout=drop_prob)
        self.ln=nn.LayerNorm(normalized_shape=seq_length,eps=1.0e-5)
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).to(device) #internal state
        x=self.ln(x.transpose(2,1))#add layer_norm
        x=x.transpose(2,1)
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = output
        return out    
class FCN(nn.Module):
    def __init__(self,c_in=Num_feature, c_out=1, layers=[128, 256, 128], kss=[7, 5, 3],embed_size1=128,embed_size2=256,heads=1,drop_prob=0):#c_in是feacture number
        super(FCN, self).__init__()
        assert len(layers) == len(kss)
        self.convblock1 = nn.Conv1d(c_in, layers[0], kss[0])
        self.convblock2 = nn.Conv1d(layers[0], layers[1], kss[1])
        self.convblock3 = nn.Conv1d(layers[1], layers[2], kss[2])
        self.gap=torch.nn.AdaptiveAvgPool1d((m+1))
        self.fc = nn.Linear((layers[2]+2*40), 10)
        self.fcc = nn.Linear(10,2)
        self.softmax=nn.Softmax(dim=1)
        self.sigmoid=nn.Sigmoid()
        self.relu=nn.ReLU()
        self.prelu=nn.PReLU()
        self.tanh=nn.Tanh()
        self.bn1=nn.BatchNorm1d(layers[0],eps=1e-03,momentum=0.99)
        self.bn2=nn.BatchNorm1d(layers[1],eps=1e-03,momentum=0.99)
        self.bn3=nn.BatchNorm1d(layers[2],eps=1e-03,momentum=0.99)
        self.dropout=nn.Dropout(p=0)
        self.lstm1 = LSTM1(num_classes=m+1, input_size=Num_feature, hidden_size=40, num_layers=2,seq_length=m+1).to(device)#input_size=feacture number
    def forward(self, x):
        x_fcn = self.convblock1(x)
        x_fcn=self.bn1(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.convblock2(x_fcn)
        x_fcn=self.bn2(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.convblock3(x_fcn)
        x_fcn=self.bn3(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.gap(x_fcn)#size(N,C,m+1)
        x_fcn=x_fcn.transpose(1,2)
        ####LSTM
        x_lstm=x.transpose(1,2)
        x_lstm=self.lstm1(x_lstm)
        ###concat
        out_concat = torch.cat((x_lstm,x_fcn),2)
        out_concat=self.prelu(out_concat)
        out_concat=self.fc(out_concat)
        out_concat=self.prelu(out_concat)
        out_concat=self.fcc(out_concat)#size(N,m+1,2)
        outt=out_concat.transpose(1,2)#size(N,2,m+1)
        out_concat=self.prelu(outt)
        out_concat=self.softmax(outt)#calculate along dim=1
        return out_concat

In [ ]:
#training
num_epochs = 800 #changable, depends on the loss history during training
learning_rate = 0.005#also changable
fcn = FCN().to(device) 
weight=torch.Tensor([1,100]).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=weight)#weighted crossentropy loss
optimizer = torch.optim.Adam(fcn.parameters(), lr=learning_rate) 
scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=50,
verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
losses=[]
for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(dataloader):  
        outputs = fcn.forward(inputs.to(device)) #forward pass
        loss = criterion(outputs, labels.to(device)) #forCrossEntropyLoss0411
        loss.backward() #calculates the loss of the loss function
        optimizer.step() #improve from loss, i.e backprop
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    if epoch % 100 == 0:
              losses.append(loss.item())
              print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
#accuracy matrix:
train_predict1 = fcn(X_test_tensors.to(device))#forward pass
train_predict1=train_predict1.cpu()
data_predict = train_predict1.data.numpy() #numpy conversion
dataY_plot = y_test_tensors.data.numpy()
data_predict1=data_predict[:,1,:]
label_predicted=np.argwhere(np.around(data_predict1)==1)
label_real1=np.argwhere(dataY_plot==1)
sum_real=0
sum_predicted=0
for i in range (label_real1.shape[0]):
    for any in data_predict1[label_real1[i][0],label_real1[i][1]-4:label_real1[i][1]+5]:
        if any>=0.5:
            sum_real=sum_real+1
            break
for j in range(label_predicted.shape[0]):
    for any in dataY_plot[label_predicted[j][0],label_predicted[j][1]-4:label_predicted[j][1]+5]:
        if any==1:
            sum_predicted=sum_predicted+1
            break    
accuracy=2/(1/(sum_real/label_real1.shape[0])+1/(sum_predicted/label_predicted.shape[0]))

In [ ]:
#figure 8 in the paper
#each accuracy point needs a training process
plt.plot([0,0.01,0.05,0.1,0.5,1,2],[98.60,99.03,93.59,89.08,79.18,75.67,67.82],"r--o",label="Injection",markersize="8",markerfacecolor='none')#99.37
plt.plot([0,0.01,0.05,0.1,0.5,1,2],[99.74,100,99.78,99.71,96.49,90.64,78.57],"g-->",label="Warmback",markersize="8",markerfacecolor='none')
plt.plot([0,0.01,0.05,0.1,0.5,1,2],[99.74,100,99.51,97.71,87.25,78.27,72.42],"b--*",label="Static",markersize="8",markerfacecolor='none')#,markeredgewidth="1")#77.96
plt.legend()
plt.xlabel("Noise standard deviation/%")
plt.ylabel("Accuracy/%")
#zoom in curve in figure 8
plt.plot([0,0.01,0.05,],[98.60,99.03,93.59],"r--o",label="injection",markersize="8",markerfacecolor='none')#99.37
plt.plot([0,0.01,0.05],[99.74,100,99.78],"g-->",label="warmback",markersize="8",markerfacecolor='none')
plt.plot([0,0.01,0.05],[99.74,100,99.51],"b--*",label="static",markersize="8",markerfacecolor='none')#77.96
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

In [ ]:
#figure 11 in the paper
#each accuracy point needs a training process
fig = plt.figure()
plt.plot([1,2,4,6],[90.64,96.58,98.60,99.24],"r-.o",label="Warmback",markersize="8")
plt.plot([0,1,2,4,6],[75.67,93.32,97.31,98.74,99.37],"g-.<",label="Injection+Warmback",markersize="8")
plt.plot([1,2,4,6],[96.47,98.13,98.88,99.37],"b-.*",label="Injection+Warmback+Static",markersize="8")#all
plt.xlabel("Number of warmback stages/[-]")
plt.ylabel("Accuracy/%")
fig.legend(loc=3, bbox_to_anchor=(0.5,0.4))#, bbox_transform=ax.transAxes)